In [12]:
import sys


def extract_input(input):
    # raw_data = sys.stdin.readline().split(" ")
    nested_size = []
    nested_matrix = []
    nested_emissions_count = 0
    nested_emissions = []
    i = 0
    #for line in sys.stdin.readlines():
    for line in input:
        # for line in input:
        i += 1
        raw_data = line.split()
        if (i == 4):
            nested_emissions = [int(x) for x in raw_data[1:]]
            nested_emissions_count = int(raw_data[0])
        else:
            nested_matrix.append([float(x) for x in raw_data[2:]])
            nested_size.append([int(x) for x in raw_data[0:2]])
    return nested_matrix, nested_size, nested_emissions, nested_emissions_count


def form_matrix(raw_matrix, size):
    final = []
    rows = size[0]
    columns = size[1]
    for i in range(rows):
        final.append(raw_matrix[columns * i: columns * (i + 1)])
    return final


def array_to_string(array):
    result = ""
    for i in range(len(array) - 1):
        result += str(array[i]) + " "
    result += str(array[len(array) - 1])
    return result


def produce_output(A, B):
    first_line = []
    first_line.append(len(A))
    first_line.append(len(A))
    for i in range(len(A)):
        for j in range(len(A)):
            first_line.append(round(A[i][j], 6))

    second_line = []
    second_line.append(len(B))
    second_line.append(len(B[0]))
    for i in range(len(B)):
        for j in range(len(B[0])):
            second_line.append(round(B[i][j], 6))

    return array_to_string(first_line), array_to_string(second_line)


def compute_probability(A, B, Pi, O, T):
    alpha_matrix, c_matrix = compute_alpha_matrix(A, B, Pi, O, T)
    N = len(A)
    probability = 0
    for i in range(N):
        probability += alpha_matrix[T - 1][i]

    return probability


def compute_beta_matrix(A, B, Pi, O, T, c_matrix):
    rows = T  # T (number of Observations)
    cols = len(A)  # N (number of states)
    beta_matrix = [[0 for _ in range(cols)] for _ in range(rows)]

    # fill in the initial states
    for j in range(cols):
        beta_matrix[T - 1][j] = c_matrix[T - 1]

    # fill the other states
    for t in range(T - 2, 0, -1):
        for i in range(cols):
            for j in range(cols):
                beta_matrix[t][i] += beta_matrix[t + 1][j] * B[j][O[t + 1]] * A[i][j]
            beta_matrix[t][i] = beta_matrix[t][i] * c_matrix[t]
    return beta_matrix


def compute_alpha_matrix(A, B, Pi, O, T):
    rows = T  # T (number of Observations)
    cols = len(A)  # N (number of states)

    alpha_matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    c_matrix = [0 for _ in range(T)]

    # rescaling factor
    c_0 = 0

    # fill in the initial states
    for j in range(cols):
        alpha_matrix[0][j] = Pi[0][j] * B[j][O[0]]
        c_0 += alpha_matrix[0][j]

    # rescale
    c_0 = 1 / c_0
    c_matrix[0] = c_0
    for j in range(cols):
        alpha_matrix[0][j] = c_0 * alpha_matrix[0][j]

    # fill the other states
    for t in range(1, rows):
        c_t = 0
        for i in range(cols):
            for j in range(cols):
                alpha_matrix[t][i] += alpha_matrix[t - 1][j] * A[j][i]
            alpha_matrix[t][i] *= B[i][O[t]]
            c_t += alpha_matrix[t][i]

        # rescale
        c_t = 1 / c_t
        c_matrix[t] = c_t
        for i in range(cols):
            alpha_matrix[t][i] = c_t * alpha_matrix[t][i]

    return alpha_matrix, c_matrix


def compute_di_gamma_function(t, i, j, A, B, O, alpha_matrix, beta_matrix):
    numerator = alpha_matrix[t][i] * A[i][j] * B[j][O[t + 1]] * beta_matrix[t + 1][j]
    denomenator = 0
    for k in range(len(A)):
        denomenator += alpha_matrix[len(O) - 1][k]
    return numerator / denomenator


def compute_gamma_function(t, i, A, B, O, alpha_matrix, beta_matrix):
    result = 0
    for j in range(len(A)):
        result += compute_di_gamma_function(t, i, j, A, B, O, alpha_matrix, beta_matrix)

    return result


# ----------------------------------------------------------------------------------------------------------------------
def compute_di_gamma_matrix(A, B, O, alpha_matrix, beta_matrix):
    T = len(O)
    N = len(A)

    # T*N*N
    di_gamma_matrix = [[[0] * N for n in range(N)] for t in range(T)]

    denom = 0
    for k in range(N):
        denom += alpha_matrix[T - 1][k]

    for t in range(T - 1):
        for i in range(N):
            for j in range(N):
                num = alpha_matrix[t][i] * A[i][j] * B[j][O[t + 1]] * beta_matrix[t + 1][j]
                di_gamma_matrix[t][i][j] = num / denom

    return di_gamma_matrix


def compute_gamma_matrix(A, B, O, alpha_matrix, beta_matrix):
    T = len(O)
    N = len(A)

    #T*N
    gamma_matrix = [[0 for _ in range(N)] for _ in range(T)]
    di_gamma_matrix = compute_di_gamma_matrix(A, B, O, alpha_matrix, beta_matrix)

    for t in range(T):
        for i in range(N):
            for j in range(N):
                gamma_matrix[t][i] += di_gamma_matrix[t][i][j]

    #    for i in range(N):
    #        gamma_matrix[T - 1][i] = alpha_matrix[T - 1][i]

    return gamma_matrix


# -----------------------------------------------------------------------------------------------------------------------

def Baum_Welch_algorithm(A, B, Pi, O, T, iter):
    N = len(A)
    K = len(B[0])

    # 2. Compute alpha, beta matrices
    alpha_matrix, c_matrix = compute_alpha_matrix(A, B, Pi, O, T)
    beta_matrix = compute_beta_matrix(A, B, Pi, O, T, c_matrix)

    # 3. Re-estimate the model
    new_A = [[0 for _ in range(N)] for _ in range(N)]
    # N*K
    new_B = [[0 for _ in range(K)] for _ in range(N)]
    new_Pi = [[0 for _ in range(N)]]

    di_gamma_matrix = compute_di_gamma_matrix(A, B, O, alpha_matrix, beta_matrix)
    gamma_matrix = compute_gamma_matrix(A, B, O, alpha_matrix, beta_matrix)

    # re-estimate A
    for i in range(N):
        for j in range(N):
            denominator = 0
            for t in range(T - 1):
                denominator += gamma_matrix[t][i]

            numerator = 0
            for t in range(T - 1):
                numerator += di_gamma_matrix[t][i][j]
            new_A[i][j] = numerator / denominator

    # re-estimate B
    for j in range(N):
        for k in range(K):
            denominator = 0
            for t in range(T - 1):
                denominator += gamma_matrix[t][j]

            numerator = 0
            for t in range(T - 1):
                if O[t] == k:
                    numerator += gamma_matrix[t][j]
            new_B[j][k] = numerator / denominator

    # re-estimate Pi
    for i in range(N):
        new_Pi[0][i] = gamma_matrix[0][i]

    iter += 1
    
    if iter < 50:
        return Baum_Welch_algorithm(new_A, new_B, new_Pi, O, T, iter)
    else:
        return new_A, new_B


def main(input):
    nested_matrix, nested_size, O, T = extract_input(input)
    A_raw, B_raw, Pi_raw = nested_matrix[0], nested_matrix[1], nested_matrix[2]
    A_size, B_size, Pi_size = nested_size[0], nested_size[1], nested_size[2]

    A, B, Pi = form_matrix(A_raw, A_size), form_matrix(B_raw, B_size), form_matrix(Pi_raw, Pi_size)


    new_A, new_B = Baum_Welch_algorithm(A, B, Pi, O, T, 0)

    output_1, output_2 = produce_output(new_A, new_B)
    sys.stdout.write(output_1)
    sys.stdout.write("\n")
    sys.stdout.write(output_2)


if __name__ == '__main__':
    input_1 = "3 3 0.800000 0.100000 0.100000 0.100000 0.800000 0.100000 0.100000 0.100000 0.800000"
    input_2 = "3 2 0.600000 0.400000 0.400000 0.600000 0.400000 0.600000"
    input_3 = "1 3 1.000000 0.000000 0.000000"
    input_4 = "1300 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 1 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 1 1 0 1 1 1 1 0 1 0 1"
    input = []
    input.append(input_1)
    input.append(input_2)
    input.append(input_3)
    input.append(input_4)
    main(input)

IndexError: list index out of range